## Librerías

In [1]:
import numpy as np
import pandas as pd
import urllib.request
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

## Data

In [2]:
train = pd.read_csv('../Kaggle1/data/train.csv')

In [3]:
train.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1104,Acer,Aspire ES1-523,Notebook,15.6,1366x768,AMD A8-Series 7410 2.2GHz,4GB,500GB HDD,AMD Radeon R5,Windows 10,2.4kg,387.0
1,114,Dell,XPS 13,Ultrabook,13.3,Quad HD+ / Touchscreen 3200x1800,Intel Core i7 7560U 2.4GHz,8GB,256GB SSD,Intel Iris Plus Graphics 640,Windows 10,1.23kg,1379.0
2,172,Acer,Aspire A517-51G,Notebook,17.3,IPS Panel Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,256GB SSD,Nvidia GeForce MX150,Windows 10,3kg,854.0
3,918,HP,Elitebook 820,Netbook,12.5,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,512GB SSD,Intel HD Graphics 620,Windows 10,1.26kg,1483.0
4,447,Lenovo,IdeaPad 320-15AST,Notebook,17.3,1600x900,AMD A6-Series 9220 2.5GHz,8GB,1TB HDD,AMD Radeon R4,Windows 10,2.8kg,519.0


In [4]:
train['Ram_clean'] = train['Ram'].map(lambda x: x.rstrip('GB'))
train['Weight_clean'] = train['Weight'].map(lambda x: x.rstrip('kg'))

In [7]:
train['GHz'] = train['Cpu'].str.findall(r'(\d+\.\d+|\d+)GHz')
train['GHz'] = train['GHz'].apply(lambda x: float(x[0]) if x else None)

In [12]:
train['PX'] = train['ScreenResolution'].str.findall(r'(\d+)x(\d+)')
train['PX_ancho'] = train['PX'].apply(lambda x: x[0][0] if x else None)
train['PX_alto'] = train['PX'].apply(lambda x: x[0][1] if x else None)
train['Marca_CPU'] = train['Cpu'].str.split().str[0]

In [22]:
train.drop(columns=['Ram', 'Weight'], inplace=True)
train.drop(columns=['ScreenResolution'], inplace=True)
train.drop(columns=['PX'], inplace=True)
train.drop(columns=['Cpu'], inplace=True)

In [25]:
train['Ram_clean'] = train['Ram_clean'].astype(int)
train['Weight_clean'] = train['Weight_clean'].astype(float)
train['PX_ancho'] = train['PX_ancho'].astype(int)
train['PX_alto'] = train['PX_alto'].astype(int)

In [32]:
train.head()

,laptop_ID,Company,Product,TypeName,Inches,Memory,Gpu,OpSys,Price_euros,Ram_clean,Weight_clean,GHz,PX_ancho,PX_alto,Marca_CPU
0,1104,Acer,Aspire ES1-523,Notebook,15.6,500GB HDD,AMD Radeon R5,Windows 10,387.0,4,2.40,2.2,1366,768,AMD
1,114,Dell,XPS 13,Ultrabook,13.3,256GB SSD,Intel Iris Plus Graphics 640,Windows 10,1379.0,8,1.23,2.4,3200,1800,Intel
2,172,Acer,Aspire A517-51G,Notebook,17.3,256GB SSD,Nvidia GeForce MX150,Windows 10,854.0,8,3.00,1.6,1920,1080,Intel
3,918,HP,Elitebook 820,Netbook,12.5,512GB SSD,Intel HD Graphics 620,Windows 10,1483.0,8,1.26,2.7,1920,1080,Intel
4,447,Lenovo,IdeaPad 320-15AST,Notebook,17.3,1TB HDD,AMD Radeon R4,Windows 10,519.0,8,2.80,2.5,1600,900,AMD


In [34]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   laptop_ID     912 non-null    int64  
 1   Company       912 non-null    object 
 2   Product       912 non-null    object 
 3   TypeName      912 non-null    object 
 4   Inches        912 non-null    float64
 5   Memory        912 non-null    object 
 6   Gpu           912 non-null    object 
 7   OpSys         912 non-null    object 
 8   Price_euros   912 non-null    float64
 9   Ram_clean     912 non-null    int32  
 10  Weight_clean  912 non-null    float64
 11  GHz           912 non-null    float64
 12  PX_ancho      912 non-null    int32  
 13  PX_alto       912 non-null    int32  
 14  Marca_CPU     912 non-null    object 
dtypes: float64(4), int32(3), int64(1), object(7)
memory usage: 96.3+ KB


## X e y

In [36]:
train2 = train[['Inches', 'Price_euros', 'Ram_clean', 'Weight_clean', 'GHz', 'PX_ancho', 'PX_alto']]

In [37]:
X = train2.drop(['Price_euros'], axis=1)
y = train2['Price_euros'].copy()

In [38]:
X.shape

(912, 6)

In [39]:
y.shape

(912,)

In [40]:
from sklearn.model_selection import train_test_split

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

## Model

In [43]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

In [44]:
model.fit(X_train, y_train)

LinearRegression()

In [46]:
pred = model.predict(X_test)

In [47]:
pred

array([1186.8041872 ,  320.00056627,  870.58209277, 1094.0640469 ,
       1718.79913564,  608.59657622,  784.9343581 , 1693.0335179 ,
        924.67867666, 1241.03725349,  862.46529853,  455.96911341,
        446.82064826, 1355.46649352, 1366.67366111, 1731.33524898,
       1861.69777002, 1267.43262799,  912.09377309,  767.18432393,
       1255.36415945,  797.47047145,  908.7012163 , 1287.7014579 ,
       1012.50130062,  569.48887143,  870.81229123, 1785.65840679,
        924.67867666,  857.15326933, 1297.2231896 ,  989.66308756,
        975.33391353, 1451.5615993 , 1205.43110141,  221.83628917,
        917.05862519, 1112.26582176, 1192.05924718, 1119.0311656 ,
       1113.64063686, 1241.84979894, 1126.0216762 , 2044.76066011,
       1157.77982999,  309.07196587, 1541.8920974 ,  465.27091666,
       1126.5528336 ,  666.93807059, 1923.5977757 , 1253.67749907,
        718.20327741, 1775.9340996 ,  834.78826871, 1201.25239696,
       1924.46550101,  897.9349682 , 2089.99089739, 1720.05274

In [49]:
from sklearn.metrics import mean_squared_error

print('RMSE :', np.sqrt(mean_squared_error(y_test, pred)))

RMSE : 444.98220486445086


## Predicción

In [50]:
X_pred = pd.read_csv('../Kaggle1/data/test.csv')
X_pred.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
0,750,Toshiba,Tecra X40-D-10G,Notebook,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.25kg
1,726,Mediacom,Smartbook 142,Notebook,14.0,IPS Panel Full HD 1920x1200,Intel Atom Z8350 1.92GHz,4GB,32GB HDD,Intel HD Graphics 400,Windows 10,1.4kg
2,633,Mediacom,SmartBook 140,Notebook,14.0,Full HD 1920x1080,Intel Atom x5-Z8350 1.44GHz,2GB,32GB Flash Storage,Intel HD Graphics,Windows 10,1.4kg
3,363,Dell,Inspiron 7577,Gaming,15.6,Full HD 1920x1080,Intel Core i5 7300HQ 2.5GHz,8GB,1TB HDD,Nvidia GeForce GTX 1050,Windows 10,2.65kg
4,319,Asus,VivoBook Flip,2 in 1 Convertible,11.6,Touchscreen 1366x768,Intel Celeron Dual Core N3350 1.1GHz,2GB,32GB Flash Storage,Intel HD Graphics 500,Windows 10,1.1kg


In [51]:
X_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         391 non-null    int64  
 1   Company           391 non-null    object 
 2   Product           391 non-null    object 
 3   TypeName          391 non-null    object 
 4   Inches            391 non-null    float64
 5   ScreenResolution  391 non-null    object 
 6   Cpu               391 non-null    object 
 7   Ram               391 non-null    object 
 8   Memory            391 non-null    object 
 9   Gpu               391 non-null    object 
 10  OpSys             391 non-null    object 
 11  Weight            391 non-null    object 
dtypes: float64(1), int64(1), object(10)
memory usage: 36.8+ KB


In [52]:
X_pred['Ram_clean'] = X_pred['Ram'].map(lambda x: x.rstrip('GB'))
X_pred['Weight_clean'] = X_pred['Weight'].map(lambda x: x.rstrip('kg'))

In [53]:
X_pred['GHz'] = X_pred['Cpu'].str.findall(r'(\d+\.\d+|\d+)GHz')
X_pred['GHz'] = X_pred['GHz'].apply(lambda x: float(x[0]) if x else None)

In [54]:
X_pred['PX'] = X_pred['ScreenResolution'].str.findall(r'(\d+)x(\d+)')
X_pred['PX_ancho'] = X_pred['PX'].apply(lambda x: x[0][0] if x else None)
X_pred['PX_alto'] = X_pred['PX'].apply(lambda x: x[0][1] if x else None)
X_pred['Marca_CPU'] = X_pred['Cpu'].str.split().str[0]

In [55]:
X_pred.drop(columns=['Ram', 'Weight'], inplace=True)
X_pred.drop(columns=['ScreenResolution'], inplace=True)
X_pred.drop(columns=['PX'], inplace=True)
X_pred.drop(columns=['Cpu'], inplace=True)

In [56]:
X_pred['Ram_clean'] = X_pred['Ram_clean'].astype(int)
X_pred['Weight_clean'] = X_pred['Weight_clean'].astype(float)
X_pred['PX_ancho'] = X_pred['PX_ancho'].astype(int)
X_pred['PX_alto'] = X_pred['PX_alto'].astype(int)

In [58]:
X_pred.head()

,laptop_ID,Company,Product,TypeName,Inches,Memory,Gpu,OpSys,Ram_clean,Weight_clean,GHz,PX_ancho,PX_alto,Marca_CPU
0,750,Toshiba,Tecra X40-D-10G,Notebook,14.0,256GB SSD,Intel HD Graphics 620,Windows 10,8,1.25,2.50,1920,1080,Intel
1,726,Mediacom,Smartbook 142,Notebook,14.0,32GB HDD,Intel HD Graphics 400,Windows 10,4,1.40,1.92,1920,1200,Intel
2,633,Mediacom,SmartBook 140,Notebook,14.0,32GB Flash Storage,Intel HD Graphics,Windows 10,2,1.40,1.44,1920,1080,Intel
3,363,Dell,Inspiron 7577,Gaming,15.6,1TB HDD,Nvidia GeForce GTX 1050,Windows 10,8,2.65,2.50,1920,1080,Intel
4,319,Asus,VivoBook Flip,2 in 1 Convertible,11.6,32GB Flash Storage,Intel HD Graphics 500,Windows 10,2,1.10,1.10,1366,768,Intel


In [60]:
X_pred = X_pred[['Inches', 'Ram_clean', 'Weight_clean', 'GHz', 'PX_ancho', 'PX_alto']]

In [61]:
predictions_submit = model.predict(X_pred)
predictions_submit

array([1189.13415407,  862.79905366,  440.13899958, 1132.82089027,
        310.90328765,  666.40771618,  536.05923585, 1780.95672392,
       1747.1092179 , 1207.69770943, 1099.80912514, 1015.67518736,
       1118.19542471,  917.05862519,  654.28947328, 1071.92509231,
        994.25966245, 1924.26504977,  558.2145484 , 1168.11330415,
       1153.84174019, 1066.59788451,  473.21045511,  904.9403823 ,
        347.04885821, 1412.83070654,  760.45698256,  827.75983191,
        801.64917589, 1207.69770943,  917.15700866, 1159.75589525,
        966.19035697, 1122.37412916, 1159.75589525, 1231.00836282,
       1159.75589525, 1392.54367614,  777.57449219, 1868.69743535,
       1731.56182291, 1126.5528336 ,  494.96291368, 1815.9021255 ,
        866.40338833, 1850.81909774, 1259.7049411 , 1731.33524898,
       1009.99407796, 1694.67593681,  752.1491669 ,  668.4970684 ,
       1115.68820204, 1046.54828215,  469.44962111,  794.96324878,
       1122.37412916,  608.59657622, 1243.12660572, 1249.81253

In [65]:
predictions_submit.shape

(391,)

## Submission

In [67]:
sample = pd.read_csv('../Kaggle1/data/sample_submission.csv')

In [68]:
sample.head()

,laptop_ID,Price_euros
0,750,500
1,726,500
2,633,500
3,363,650
4,319,650


In [69]:
submission = pd.DataFrame({'laptop_ID': sample['laptop_ID'], 'Price_euros': predictions_submit})

In [70]:
submission.head()

,laptop_ID,Price_euros
0,750,1189.134154
1,726,862.799054
2,633,440.139000
3,363,1132.820890
4,319,310.903288


In [71]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.laptop_ID.all() == sample.laptop_ID.all():
                print("You're ready to submit!")
                submission.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto de Clara: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `diamonds_test.csv`. Lloro.")

In [72]:
chequeator(submission)

You're ready to submit!
